# 使用预训练词嵌入模型

## 1. 为什么使用预训练词嵌入？

### 1.1 从头训练 vs 预训练

| 维度 | 从头训练 | 预训练词嵌入 |
|------|---------|-------------|
| **训练数据** | 任务相关的标注数据 | 大规模无标注语料（数十亿词）|
| **语义知识** | 仅学到任务相关的模式 | 包含丰富的通用语义知识 |
| **数据需求** | 需要大量标注数据（>100K） | 标注数据要求低（<10K）|
| **训练时间** | 较长 | 较短（只需微调）|
| **小数据集表现** | 差（过拟合） | 好（迁移学习）|
| **专业领域** | 好（领域适配） | 一般（可能不适配）|

### 1.2 常见的预训练词嵌入

#### Word2Vec (2013, Google)
- **训练方法**：Skip-gram / CBOW
- **训练语料**：Google News (1000亿词)
- **维度**：通常300维
- **特点**：捕捉语义和句法关系

#### GloVe (2014, Stanford)
- **全称**：Global Vectors for Word Representation
- **训练方法**：基于全局词共现统计
- **训练语料**：
  - Wikipedia + Gigaword (60亿词)
  - Common Crawl (420亿 / 840亿词)
- **维度**：50, 100, 200, 300维
- **优势**：结合了全局统计和局部上下文

#### FastText (2016, Facebook)
- **创新**：利用子词信息（n-gram）
- **优势**：
  - 可以为OOV词生成向量
  - 对拼写错误鲁棒
  - 适合形态丰富的语言

### 1.3 本教程使用GloVe

我们将使用GloVe 840B（8400亿token，300维）预训练向量，原因：
1. 训练语料规模大，覆盖词汇广
2. 性能优秀，广泛使用
3. 容易获取和使用

## 2. 任务说明

**任务**：IMDB电影评论情感分类（二分类）

**挑战**：
- 使用原始IMDB数据集（而非Keras预处理版本）
- 从头构建tokenizer和词汇表
- 加载GloVe向量并映射到词汇表
- 对比预训练嵌入 vs 随机初始化的效果

**数据集说明**：
- 50,000条电影评论（25,000训练 + 25,000测试）
- 二分类：正面(pos)评论 vs 负面(neg)评论
- 每个评论是一段英文文本

## 3. 数据加载与预处理

### 3.1 读取原始文本文件

IMDB数据集通常组织为以下结构：
```
imdb/
  train/
    pos/  # 正面评论
      0.txt
      1.txt
      ...
    neg/  # 负面评论
      0.txt
      1.txt
      ...
  test/
    pos/
    neg/
```

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os

# 设置随机种子以确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("=" * 70)
print("使用Keras内置IMDB数据集")
print("=" * 70)
print("\n注意：本示例使用Keras内置的IMDB数据集")
print("如果需要使用自定义数据集，请修改数据加载部分\n")

# 从Keras加载IMDB数据集的索引
from tensorflow.keras.datasets import imdb

# 获取词汇表映射（word -> index）
word_index = imdb.get_word_index()

# Keras的IMDB数据集索引从3开始
# 0: padding, 1: start token, 2: unknown token
# 我们需要调整索引
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2

# 创建反向映射（index -> word）
reverse_word_index = {v: k for k, v in word_index.items()}

# 加载数据（这里加载已经编码为整数的数据）
(x_train_sequences, y_train), (x_test_sequences, y_test) = imdb.load_data()

print(f"训练集大小: {len(x_train_sequences)}")
print(f"测试集大小: {len(x_test_sequences)}")
print(f"词汇表大小: {len(word_index)}")

# 将整数序列转换回文本（用于后续tokenizer）
def decode_review(encoded_review):
    """将整数序列解码为文本"""
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_review])

# 重构文本列表
texts_train = [decode_review(seq) for seq in x_train_sequences]
texts_test = [decode_review(seq) for seq in x_test_sequences]

print(f"\n第一条评论示例：")
print(f"原始长度: {len(x_train_sequences[0])} 个词")
print(f"文本内容: {texts_train[0][:200]}...")  # 显示前200个字符
print(f"标签: {y_train[0]} ({'正面' if y_train[0] == 1 else '负面'})")

### 3.2 文本分词和序列化

**核心步骤**：
1. **Tokenization**：将文本分词并构建词汇表
2. **Sequence Encoding**：将文本转换为整数序列
3. **Padding**：统一序列长度

In [ ]:
print("=" * 70)
print("文本分词和序列化")
print("=" * 70)

# 设置超参数
max_words = 10000  # 只保留最常见的10,000个词
maxlen = 100       # 每条评论最多100个词

# 创建Tokenizer
tokenizer = Tokenizer(num_words=max_words)

# 在训练数据上拟合tokenizer（构建词汇表）
print("\n正在构建词汇表...")
tokenizer.fit_on_texts(texts_train)

# 将文本转换为整数序列
sequences_train = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

# 获取词汇表
word_index = tokenizer.word_index
print(f"完整词汇表大小: {len(word_index)}")
print(f"保留的词汇表大小: {min(max_words, len(word_index))}")

# 显示前10个最常见的词
print("\n最常见的10个词：")
sorted_words = sorted(word_index.items(), key=lambda x: x[1])[:10]
for word, idx in sorted_words:
    print(f"  {word:15s} -> 索引 {idx}")

# 序列填充
print(f"\n正在进行序列填充（maxlen={maxlen}）...")
x_train = pad_sequences(sequences_train, maxlen=maxlen)
x_test = pad_sequences(sequences_test, maxlen=maxlen)

print(f"\n训练数据形状: {x_train.shape}")
print(f"测试数据形状: {x_test.shape}")
print(f"  - {x_train.shape[0]} 条训练评论")
print(f"  - 每条评论 {x_train.shape[1]} 个词")

### 3.3 划分训练集和验证集

从训练集中划分出一部分作为验证集，用于：
- 监控训练过程
- 早停（Early Stopping）
- 超参数调优

In [ ]:
print("=" * 70)
print("数据集划分")
print("=" * 70)

# 设置训练集和验证集的大小
training_samples = 20000   # 用于训练的样本数
validation_samples = 5000  # 用于验证的样本数

# 打乱数据顺序（保证随机性）
indices = np.arange(x_train.shape[0])
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

# 划分训练集和验证集
x_train_final = x_train[:training_samples]
y_train_final = y_train[:training_samples]

x_val = x_train[training_samples:training_samples + validation_samples]
y_val = y_train[training_samples:training_samples + validation_samples]

print(f"训练集: {len(x_train_final)} 条")
print(f"验证集: {len(x_val)} 条")
print(f"测试集: {len(x_test)} 条")

# 统计标签分布
print(f"\n训练集标签分布：")
print(f"  正面: {np.sum(y_train_final == 1)} ({np.mean(y_train_final == 1):.1%})")
print(f"  负面: {np.sum(y_train_final == 0)} ({np.mean(y_train_final == 0):.1%})")

## 4. 加载GloVe预训练词向量

### 4.1 GloVe文件格式

GloVe文件是一个文本文件，每行包含一个词和它的向量：
```
the -0.038194 -0.24487 0.72812 ... (300个数字)
of -0.071953 0.23127 0.023731 ... (300个数字)
and -0.026165 -0.12684 0.053322 ... (300个数字)
```

### 4.2 加载策略

1. 读取GloVe文件
2. 构建字典：{word: vector}
3. 只保留我们词汇表中出现的词
4. 未在GloVe中出现的词使用零向量或随机初始化

In [ ]:
print("=" * 70)
print("加载GloVe预训练词向量")
print("=" * 70)

# GloVe文件路径（相对于当前notebook）
glove_dir = "GloVe"
glove_file = "glove.840B.300d.txt"
glove_path = os.path.join(glove_dir, glove_file)

print(f"\n正在加载GloVe向量: {glove_path}")
print("这可能需要几分钟时间...\n")

# 存储词向量的字典
embeddings_index = {}

# 统计信息
line_count = 0
error_count = 0

try:
    with open(glove_path, encoding='utf-8') as f:
        for line in f:
            line_count += 1
            
            # 每10万行打印一次进度
            if line_count % 100000 == 0:
                print(f"  已处理 {line_count:,} 行...")
            
            # 解析每一行
            values = line.split()
            word = values[0]  # 第一个元素是词
            
            try:
                # 剩余元素是词向量（转换为float32以节省内存）
                coefs = np.asarray(values[1:], dtype='float32')
                
                # 验证向量维度（应该是300）
                if len(coefs) == 300:
                    embeddings_index[word] = coefs
                else:
                    error_count += 1
                    
            except (ValueError, IndexError) as e:
                error_count += 1
                if error_count <= 5:  # 只打印前5个错误
                    print(f"  警告：第 {line_count} 行解析失败: {str(e)[:50]}")
    
    print(f"\n加载完成！")
    print(f"  总行数: {line_count:,}")
    print(f"  成功加载: {len(embeddings_index):,} 个词向量")
    print(f"  错误行数: {error_count:,}")
    print(f"  向量维度: 300")
    
except FileNotFoundError:
    print(f"\n错误：找不到GloVe文件: {glove_path}")
    print("\n请确保GloVe文件已下载并放置在正确的路径。")
    print("下载地址: https://nlp.stanford.edu/projects/glove/")
    raise

### 4.3 构建嵌入矩阵

**目标**：创建一个嵌入矩阵 E ∈ R^(V × d)，其中：
- V = 词汇表大小（max_words）
- d = 嵌入维度（300）

**步骤**：
1. 初始化零矩阵 (max_words × 300)
2. 对于词汇表中的每个词：
   - 如果词在GloVe中 → 使用GloVe向量
   - 如果词不在GloVe中 → 保持零向量（或随机初始化）

In [ ]:
print("=" * 70)
print("构建嵌入矩阵")
print("=" * 70)

embedding_dim = 300  # GloVe向量维度

# 初始化嵌入矩阵（全零）
embedding_matrix = np.zeros((max_words, embedding_dim))

# 统计信息
found_count = 0
missing_count = 0
missing_words = []

print(f"\n正在构建嵌入矩阵 ({max_words} × {embedding_dim})...\n")

# 遍历词汇表中的每个词
for word, i in word_index.items():
    # 只处理前max_words个词
    if i < max_words:
        # 从GloVe中查找该词的向量
        embedding_vector = embeddings_index.get(word)
        
        if embedding_vector is not None:
            # 找到了，使用GloVe向量
            embedding_matrix[i] = embedding_vector
            found_count += 1
        else:
            # 未找到，保持零向量
            missing_count += 1
            if len(missing_words) < 10:  # 记录前10个缺失的词
                missing_words.append(word)

# 统计报告
total_words = min(max_words, len(word_index))
coverage = found_count / total_words * 100

print(f"嵌入矩阵构建完成！")
print(f"  词汇表大小: {total_words:,}")
print(f"  找到GloVe向量: {found_count:,} ({coverage:.1f}%)")
print(f"  未找到GloVe向量: {missing_count:,} ({100-coverage:.1f}%)")

if missing_words:
    print(f"\n未找到向量的词示例：{', '.join(missing_words[:10])}")

print(f"\n嵌入矩阵形状: {embedding_matrix.shape}")
print(f"嵌入矩阵内存占用: {embedding_matrix.nbytes / 1024 / 1024:.2f} MB")

## 5. 构建模型

### 5.1 模型架构

我们构建一个简单的分类模型：

```
输入 (batch_size, 100)
  ↓
Embedding (batch_size, 100, 300)  ← 加载GloVe权重
  ↓
Flatten (batch_size, 30000)
  ↓
Dense(32, relu) (batch_size, 32)
  ↓
Dense(1, sigmoid) (batch_size, 1)
```

### 5.2 关键设计决策

**是否冻结Embedding层？**

| 策略 | 适用场景 | 优势 | 劣势 |
|------|---------|------|------|
| **冻结**（trainable=False） | 数据量少(<10K) | 防止过拟合；训练快 | 无法适配任务 |
| **微调**（trainable=True） | 数据量中等(10K-100K) | 任务适配；性能更好 | 可能过拟合 |

本例中我们**冻结**Embedding层，因为：
1. GloVe向量已经包含丰富的语义信息
2. 防止过拟合（训练样本有限）
3. 训练速度更快

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

print("=" * 70)
print("构建模型")
print("=" * 70)

model = Sequential([
    # Embedding层：使用GloVe预训练权重
    Embedding(
        input_dim=max_words,      # 词汇表大小
        output_dim=embedding_dim,  # 嵌入维度(300)
        input_length=maxlen,       # 输入序列长度(100)
        name='embedding'
    ),
    
    # Flatten层：展平为1D向量
    Flatten(name='flatten'),
    
    # 全连接层
    Dense(32, activation='relu', name='dense1'),
    
    # 输出层
    Dense(1, activation='sigmoid', name='output')
])

# 加载GloVe权重到Embedding层
print("\n正在加载GloVe权重到Embedding层...")
model.layers[0].set_weights([embedding_matrix])

# 冻结Embedding层（不允许训练）
model.layers[0].trainable = False

print("Embedding层已冻结（trainable=False）")
print("\n模型结构：")
model.summary()

# 分析可训练参数
total_params = model.count_params()
trainable_params = sum([np.prod(v.shape) for v in model.trainable_weights])
non_trainable_params = total_params - trainable_params

print(f"\n参数统计：")
print(f"  总参数: {total_params:,}")
print(f"  可训练参数: {trainable_params:,}")
print(f"  不可训练参数: {non_trainable_params:,} (Embedding层)")

### 5.3 编译模型

In [ ]:
print("=" * 70)
print("编译模型")
print("=" * 70)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n模型编译完成！")
print("  优化器: Adam")
print("  损失函数: Binary Crossentropy")
print("  评估指标: Accuracy")

## 6. 训练模型

### 训练配置

- **epochs**: 10（为快速演示，实际可能需要更多）
- **batch_size**: 32
- **validation_data**: 使用验证集监控性能

In [ ]:
print("=" * 70)
print("训练模型")
print("=" * 70)

history = model.fit(
    x_train_final, y_train_final,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)

print("\n训练完成！")

## 7. 评估模型

In [ ]:
print("=" * 70)
print("模型评估")
print("=" * 70)

# 在测试集上评估
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)

print(f"\n测试集性能：")
print(f"  损失: {test_loss:.4f}")
print(f"  准确率: {test_acc:.4f} ({test_acc*100:.2f}%)")

# 在验证集上的最终性能
val_loss, val_acc = model.evaluate(x_val, y_val, verbose=0)
print(f"\n验证集性能：")
print(f"  损失: {val_loss:.4f}")
print(f"  准确率: {val_acc:.4f} ({val_acc*100:.2f}%)")

## 8. 可视化训练过程

In [ ]:
import matplotlib.pyplot as plt

print("=" * 70)
print("可视化训练曲线")
print("=" * 70)

# 提取训练历史
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(acc) + 1)

# 创建图表
plt.figure(figsize=(14, 5))

# 准确率曲线
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, 'bo-', label='Training Accuracy', linewidth=2)
plt.plot(epochs_range, val_acc, 'ro-', label='Validation Accuracy', linewidth=2)
plt.title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# 损失曲线
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, 'bo-', label='Training Loss', linewidth=2)
plt.plot(epochs_range, val_loss, 'ro-', label='Validation Loss', linewidth=2)
plt.title('Training and Validation Loss', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印最佳性能
best_val_acc = max(val_acc)
best_epoch = val_acc.index(best_val_acc) + 1

print(f"\n最佳验证准确率: {best_val_acc:.4f} (Epoch {best_epoch})")

## 9. 预训练嵌入 vs 随机初始化对比

### 实验设计

为了展示预训练嵌入的优势，我们训练一个随机初始化的对比模型：

| 特性 | 预训练GloVe | 随机初始化 |
|------|------------|----------|
| 初始权重 | GloVe向量 | 随机值 |
| trainable | False | True |
| 语义知识 | 包含 | 需要学习 |

**预期结果**：
- 预训练模型收敛更快
- 预训练模型泛化性能更好（验证集准确率更高）
- 随机初始化可能过拟合

In [ ]:
print("=" * 70)
print("对比实验：随机初始化 vs 预训练GloVe")
print("=" * 70)

# 构建随机初始化模型
model_random = Sequential([
    Embedding(
        input_dim=max_words,
        output_dim=embedding_dim,
        input_length=maxlen,
        name='embedding_random'
    ),
    Flatten(name='flatten_random'),
    Dense(32, activation='relu', name='dense1_random'),
    Dense(1, activation='sigmoid', name='output_random')
])

# 注意：这里不加载GloVe权重，也不冻结Embedding层
# Embedding层会从随机初始化开始训练

model_random.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n随机初始化模型结构：")
model_random.summary()

print("\n开始训练随机初始化模型...")
history_random = model_random.fit(
    x_train_final, y_train_final,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val),
    verbose=1
)

print("\n训练完成！")

### 对比分析

In [ ]:
print("=" * 70)
print("性能对比")
print("=" * 70)

# 在测试集上评估两个模型
test_loss_glove, test_acc_glove = model.evaluate(x_test, y_test, verbose=0)
test_loss_random, test_acc_random = model_random.evaluate(x_test, y_test, verbose=0)

print(f"\n测试集性能对比：")
print(f"{'':<20s} {'GloVe预训练':<15s} {'随机初始化':<15s} {'差异':<10s}")
print("-" * 65)
print(f"{'准确率':<20s} {test_acc_glove:< 15.4f} {test_acc_random:<15.4f} {test_acc_glove - test_acc_random:+.4f}")
print(f"{'损失':<20s} {test_loss_glove:<15.4f} {test_loss_random:<15.4f} {test_loss_glove - test_loss_random:+.4f}")

# 可视化对比
plt.figure(figsize=(14, 5))

# 验证准确率对比
plt.subplot(1, 2, 1)
plt.plot(history.history['val_accuracy'], 'b-', label='GloVe Pre-trained', linewidth=2)
plt.plot(history_random.history['val_accuracy'], 'r--', label='Random Init', linewidth=2)
plt.title('Validation Accuracy Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

# 验证损失对比
plt.subplot(1, 2, 2)
plt.plot(history.history['val_loss'], 'b-', label='GloVe Pre-trained', linewidth=2)
plt.plot(history_random.history['val_loss'], 'r--', label='Random Init', linewidth=2)
plt.title('Validation Loss Comparison', fontsize=14, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n关键观察：")
if test_acc_glove > test_acc_random:
    improvement = (test_acc_glove - test_acc_random) / test_acc_random * 100
    print(f"  ✓ GloVe预训练模型性能更优，提升 {improvement:.1f}%")
    print(f"  ✓ 预训练嵌入包含的语义知识有助于泛化")
else:
    print(f"  ⚠ 在此次运行中，随机初始化表现更好")
    print(f"  ⚠ 可能原因：训练数据充足，模型容量足够")

## 10. 总结与最佳实践

### 核心要点

1. **预训练词嵌入的优势**：
   - 包含在大规模语料上学到的通用语义知识
   - 提升小数据集上的性能
   - 加快训练收敛速度
   - 提高模型泛化能力

2. **何时使用预训练嵌入**：
   - 标注数据有限（<10K样本）
   - 通用NLP任务（非专业领域）
   - 需要快速原型开发
   - 词汇覆盖率高（>80%）

3. **何时从头训练**：
   - 标注数据充足（>100K样本）
   - 专业领域（医疗、法律等）
   - 预训练嵌入覆盖率低
   - 需要特定任务适配

### 实践建议

#### 选择合适的预训练模型

| 模型 | 优势 | 适用场景 |
|------|------|----------|
| **Word2Vec** | 训练快，效果好 | 通用任务 |
| **GloVe** | 全局统计，性能稳定 | 通用任务，本教程推荐 |
| **FastText** | 支持OOV，子词信息 | 形态丰富的语言，专业术语多 |
| **BERT/GPT嵌入** | 上下文相关，SOTA性能 | 对性能要求高的任务 |

#### 调优策略

1. **嵌入层是否可训练**：
   ```python
   # 小数据集：冻结
   embedding_layer.trainable = False
   
   # 中等数据集：微调（小学习率）
   embedding_layer.trainable = True
   optimizer = Adam(learning_rate=1e-4)  # 降低学习率
   
   # 大数据集：自由训练
   embedding_layer.trainable = True
   ```

2. **处理OOV词**：
   - 使用`<UNK>`标记
   - 为OOV词随机初始化向量
   - 使用FastText（利用子词信息）
   - 使用字符级或子词级模型

3. **降低内存占用**：
   - 使用float16代替float32
   - 只加载需要的词向量
   - 降低嵌入维度（如300→100）

### 进阶方向

1. **上下文相关嵌入**：
   - ELMo：双向LSTM
   - BERT：Transformer编码器
   - GPT：Transformer解码器

2. **多语言嵌入**：
   - MUSE：跨语言词嵌入
   - XLM-R：多语言预训练模型

3. **领域适配**：
   - 在领域语料上微调预训练嵌入
   - 使用领域特定的预训练模型（BioBERT、SciBERT等）

### 常见问题

**Q: GloVe向量覆盖率低怎么办？**
- A: 尝试FastText（子词级），或在领域语料上训练Word2Vec

**Q: 如何选择嵌入维度？**
- A: 标准选择300维；小数据集可用100维；大词汇表可用500维

**Q: 预训练嵌入效果不好？**
- A: 检查词汇覆盖率；考虑领域差异；尝试微调或从头训练

**Q: 如何更新嵌入层？**
- A: 设置`trainable=True`并使用较小的学习率

## 参考资源

- **GloVe**: https://nlp.stanford.edu/projects/glove/
- **Word2Vec**: https://code.google.com/archive/p/word2vec/
- **FastText**: https://fasttext.cc/
- **论文**:
  - Pennington et al. (2014): "GloVe: Global Vectors for Word Representation"
  - Mikolov et al. (2013): "Efficient Estimation of Word Representations in Vector Space"
  - Bojanowski et al. (2017): "Enriching Word Vectors with Subword Information"